#### Import packages and data

In [2104]:
import pandas as pd
import numpy as np
import seaborn as sns
import time

from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline

from scipy import stats 

In [2105]:
business=pd.read_csv('C:/Users/marast01/Desktop/PY/yelp_business_85001-85099.csv')
demographics=pd.read_csv('C:/Users/marast01/Desktop/PY/Census.csv')

In [2106]:
business.head(2)

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
0,FYWN1wneV18bWNgQjJ2GNg,"""Dental by Design""",NaN,"""4855 E Warner Rd, Ste B9""",Ahwatukee,AZ,85044,33.330690,-111.978599,4.0,22,1,Dentists;General Dentistry;Health & Medical;Or...
1,KQPW8lFf1y5BT2MxiSZ3QA,"""Western Motor Vehicle""",NaN,"""6025 N 27th Ave, Ste 1""",Phoenix,AZ,85017,33.524903,-112.115310,1.5,18,1,Departments of Motor Vehicles;Public Services ...


#### Generate new column for single category

In [2107]:
business['cat'] = "NA"

#### Generate Dummy Variable Columns

In [2108]:
list_dv = ['Fast Food','Bars','Gluten-Free','Vegetarian','Vegan','Pizza']

In [2109]:
for i in list_dv: 
    business[i] = business['categories'].str.contains(i, regex=False)

In [2110]:
business.head(2)

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,cat,Fast Food,Bars,Gluten-Free,Vegetarian,Vegan,Pizza
0,FYWN1wneV18bWNgQjJ2GNg,"""Dental by Design""",NaN,"""4855 E Warner Rd, Ste B9""",Ahwatukee,AZ,85044,33.330690,-111.978599,4.0,22,1,Dentists;General Dentistry;Health & Medical;Or...,NA,False,False,False,False,False,False
1,KQPW8lFf1y5BT2MxiSZ3QA,"""Western Motor Vehicle""",NaN,"""6025 N 27th Ave, Ste 1""",Phoenix,AZ,85017,33.524903,-112.115310,1.5,18,1,Departments of Motor Vehicles;Public Services ...,NA,False,False,False,False,False,False


#### Filter by restaurants

In [2111]:
business['temp'] = "0"
business['temp'] = business['categories'].str.contains('Restaurant', regex=False)
business = business[business.temp == True]
business['temp'].value_counts()

True    3652
Name: temp, dtype: int64

#### Categorize a restaurant based on categories

In [2112]:
list_cat = ['Nightlife','Music Venues','Venues & Event Spaces','Arts & Entertainment','Grocery',
            
            #######
            
            #generic categories go first
            'Comfort Food',
            'Bars',
            'Fast Food',
            'Breakfast & Brunch',
            'Desserts',           
            'Cafes','Bakeries','Delis','Cafeteria',
            'American (Traditional)',
            'American (New)',
            'Seafood',
            'Buffets',
            'Gluten-Free','Vegetarian', 'Vegan',
            'Cheesesteaks','Chicken',
            'Hot Dogs',
            
            #Filter through American food
            'Sandwiches','Burgers','Sports Bars','Barbeque','Steakhouses',
            #Filter through central American food
            'Latin America','Mexican','Tex-Mex','Peruvian','Salvadoran',
            #Filter through Mediterranean food
            'Mediterranean','Greek',
            #Filter through Asian Food
            'Chinese','Asian Fusion','Thai','Vietnamese','Japanese','Sushi',
            'Korean',
            #Filter through Middle East
            'Middle Eastern',
            
            #Filter through pizza and Italian
            'Pizza','Italian',
            #Filter through Carribean
            'Carribean','Caribbean','Cajun','Cuban',
            #Filter through European 
            'Modern European','French',
            'Russian','Ukranian','Polish','Uzbek','Afghan',
            'German','Bavarian',
            'Spanish',
            #Filter through African
            'African','Ethiopian',
            #Filter throuh Indian
            'Indian',#'Pakistani',
                     
            #Filter through Southern
            'Southern','Diners','Soul Food',
            #Filter through bar restaurants
            'Sports Bar','Wine Bars','Gastropub','Cocktail Bars','Dive Bar',
            #Filter through misc.
            'Coffee & Tea','Juice Bars & Smoothies','Ice Cream & Frozen Yogurt',
            'Bagels', 'Donuts',
            'Fish & Chips','Kosher','Pretzels',
            'Hawaiian', 
            
            ########
            
            #Label things that are not restaurants
            
            'Festival','Food Trucks','Event Planning & Services','Grocery', 
            'Food Delivery Services','Shopping','Active Life', 'Party & Event Planning',
            'Health Markets','Convenience Stores', 'Arcades','Professional Services',
            'Home & Garden','Dance Clubs', 'Wholesalers','Restaurant Supplies',
            'Automotive','Furniture Stores', 'Health & Medical','Gas Stations',
            'Kitchen & Bath','Beauty & Spas', 'Home Services','Appliances','Day Spas',
            'Personal Chefs','Caterers', 'Fashion','Food Court','Veterinarians'
            
             ]

In [2113]:
for i in list_cat: 
    business['temp'] = "0"
    business['temp'] = business['categories'].str.contains(i, regex=False)
    business['cat'] = np.where(business['temp'] == True, i, business['cat'])

In [2114]:
business['cat'].value_counts()

Mexican                      513
Sandwiches                   268
Burgers                      264
Italian                      217
Pizza                        195
Chinese                      164
Caterers                     117
American (Traditional)       115
American (New)                94
Coffee & Tea                  92
Sports Bar                    92
Sushi                         75
Tex-Mex                       61
Diners                        60
Greek                         59
Fast Food                     53
Barbeque                      52
Food Trucks                   52
Wine Bars                     50
Ice Cream & Frozen Yogurt     46
Chicken                       44
Thai                          43
Cocktail Bars                 43
Steakhouses                   40
Vietnamese                    38
Juice Bars & Smoothies        34
Indian                        34
Asian Fusion                  32
Japanese                      31
Grocery                       27
Bagels    

#### Remove non-restaurant categories that still remain

In [2115]:
list_drop = ['Appliances','Home Services','Festival','Health & Medical',
             'Gas Stations','Day Spas','Restaurant Supplies',
             'Personal Chefs','Veterinarians','Arts & Entertainment',
             'Kitchen & Bath','Nightlife','Furniture Stores','Beauty & Spas',
             'Arcades','Dance Clubs','Party & Event Planning',
             'Convenience Stores','Caterers','Food Trucks','Active Life',
             'Food Delivery Services','Health Markets','Shopping','Grocery',
             'Event Planning & Services','Fashion'
            ]

In [2116]:
for i in list_drop:
    business = business.loc[business['cat'] != i]

In [2117]:
business.shape[0]

3318

In [2118]:
pd.set_option('display.max_row', 100)
business['cat'].value_counts()

Mexican                      513
Sandwiches                   268
Burgers                      264
Italian                      217
Pizza                        195
Chinese                      164
American (Traditional)       115
American (New)                94
Sports Bar                    92
Coffee & Tea                  92
Sushi                         75
Tex-Mex                       61
Diners                        60
Greek                         59
Fast Food                     53
Barbeque                      52
Wine Bars                     50
Ice Cream & Frozen Yogurt     46
Chicken                       44
Thai                          43
Cocktail Bars                 43
Steakhouses                   40
Vietnamese                    38
Indian                        34
Juice Bars & Smoothies        34
Asian Fusion                  32
Japanese                      31
Bagels                        27
Mediterranean                 26
Middle Eastern                26
NA        

#### Categorize a restaurant by its name

In [2119]:
business['temp'] = business['name'].str.contains('Burger King', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Burger', business['cat'])

In [2120]:
business['temp'] = business['name'].str.contains('McDonalds', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Burger', business['cat'])

In [2121]:
business['temp'] = business['name'].str.contains('Dairy Queen', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Ice Cream & Frozen Yogurt', business['cat'])

In [2122]:
business['temp'] = business['name'].str.contains('Noodles & Company', regex=False)
business['cat'] = np.where(business['temp'] == True, 'American (New)', business['cat'])

In [2123]:
business['temp'] = business['name'].str.contains('The Loaded Potato', regex=False)
business['cat'] = np.where(business['temp'] == True, 'American (New)', business['cat'])

In [2124]:
business['temp'] = business['name'].str.contains('Food Court', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Food Court', business['cat'])

In [2125]:
business['temp'] = business['name'].str.contains('Sonic', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Ice Cream & Frozen Yogurt', business['cat'])

In [2126]:
business['temp'] = business['name'].str.contains('Mighty Mikes', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Burger', business['cat'])

In [2127]:
business['temp'] = business['name'].str.contains('Salad', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Salad', business['cat'])

In [2128]:
business['temp'] = business['name'].str.contains('Pizza', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Pizza', business['cat'])

In [2129]:
business['temp'] = business['name'].str.contains('pizza', regex=False)
business['cat'] = np.where(business['temp'] == True, 'pizza', business['cat'])

In [2130]:
business['temp'] = business['name'].str.contains('Taco', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Mexican', business['cat'])

In [2131]:
business['temp'] = business['name'].str.contains('taco', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Mexican', business['cat'])

In [2132]:
business['temp'] = business['name'].str.contains('Carl', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Burgers', business['cat'])

In [2133]:
business['temp'] = business['name'].str.contains('Red Robin', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Burgers', business['cat'])

In [2134]:
business['temp'] = business['name'].str.contains(r'(?=.*Jack)(?=.*Box)',regex=True)
business['cat'] = np.where(business['temp'] == True, 'Burgers', business['cat'])

In [2135]:
business['temp'] = business['name'].str.contains(r'(?=.*Jack)(?=.*box)',regex=True)
business['cat'] = np.where(business['temp'] == True, 'Burgers', business['cat'])

In [2136]:
business['temp'] = business['name'].str.contains('Chicken', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Chicken', business['cat'])

In [2137]:
business['temp'] = business['name'].str.contains('Fish', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Seafood', business['cat'])

In [2138]:
business['temp'] = business['name'].str.contains('Fish & Chips', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Fish & Chips', business['cat'])

In [2139]:
business['temp'] = business['name'].str.contains('Wong', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Chinese', business['cat'])

In [2140]:
business['temp'] = business['name'].str.contains('Grill', regex=False)
business['cat'] = np.where(business['temp'] == True, 'American (New)', business['cat'])

In [2141]:
business['temp'] = business['name'].str.contains('Cafe', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Cafe', business['cat'])

In [2142]:
business['temp'] = business['name'].str.contains('Deli', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Deli', business['cat'])

In [2143]:
business['temp'] = business['name'].str.contains('Pretzels', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Pretzels', business['cat'])

In [2144]:
business['temp'] = business['name'].str.contains('Nueva', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Mexican', business['cat'])

In [2145]:
business['temp'] = business['name'].str.contains('Pueblo', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Mexican', business['cat'])

In [2146]:
business['temp'] = business['name'].str.contains('La ', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Mexican', business['cat'])

In [2147]:
pd.set_option('display.max_row', 100)
business['cat'].value_counts()

Mexican                      509
American (New)               323
Pizza                        279
Burgers                      259
Sandwiches                   226
Chinese                      167
Cafe                         161
Italian                      115
Coffee & Tea                  70
Sushi                         67
Chicken                       60
American (Traditional)        58
Ice Cream & Frozen Yogurt     57
Diners                        51
Barbeque                      49
Greek                         48
Sports Bar                    47
Thai                          42
Cocktail Bars                 41
Burger                        40
Deli                          38
Wine Bars                     38
Vietnamese                    37
Steakhouses                   37
Indian                        29
Juice Bars & Smoothies        27
Japanese                      27
Asian Fusion                  25
Seafood                       22
Bagels                        20
Hot Dogs  

#### Consolidate categories

In [2148]:
business['temp'] = business['cat'].str.contains('Bavarian', regex=False)
business['cat'] = np.where(business['temp'] == True, 'German', business['cat'])

In [2149]:
business['temp'] = business['cat'].str.contains('Salvadoran', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Latin America', business['cat'])

In [2150]:
business['temp'] = business['cat'].str.contains('Peruvian', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Latin America', business['cat'])

In [2151]:
business['temp'] = business['cat'].str.contains('Ethiopian', regex=False)
business['cat'] = np.where(business['temp'] == True, 'African', business['cat'])

In [2152]:
business['temp'] = business['cat'].str.contains('Cheesesteaks', regex=False)
business['cat'] = np.where(business['temp'] == True, 'Sandwiches', business['cat'])

In [2153]:
business['cat'].value_counts()

Mexican                      509
American (New)               323
Pizza                        279
Burgers                      259
Sandwiches                   227
Chinese                      167
Cafe                         161
Italian                      115
Coffee & Tea                  70
Sushi                         67
Chicken                       60
American (Traditional)        58
Ice Cream & Frozen Yogurt     57
Diners                        51
Barbeque                      49
Greek                         48
Sports Bar                    47
Thai                          42
Cocktail Bars                 41
Burger                        40
Wine Bars                     38
Deli                          38
Steakhouses                   37
Vietnamese                    37
Indian                        29
Juice Bars & Smoothies        27
Japanese                      27
Asian Fusion                  25
Seafood                       22
Bagels                        20
Hot Dogs  

#### National chains (https://www.restaurantbusinessonline.com/top-500-chains)

In [2154]:
business['chain'] = 'Non-Chain'

In [2155]:
business['temp'] = business['name'].str.contains('In-N-Out Burger', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2156]:
business['temp'] = business['name'].str.contains('Chili', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2157]:
business['temp'] = business['name'].str.contains('Macaroni Grill', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2158]:
business['temp'] = business['name'].str.contains('Boston Market', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2159]:
business['temp'] = business['name'].str.contains('Red Robin', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2160]:
business['temp'] = business['name'].str.contains('Samurai Sam', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2161]:
business['temp'] = business['name'].str.contains('Smashburger', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2162]:
business['temp'] = business['name'].str.contains('Rosati', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2163]:
business['temp'] = business['name'].str.contains('Filiberto', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2164]:
business['temp'] = business['name'].str.contains('Hooters', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2165]:
business['temp'] = business['name'].str.contains('Papa John', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2166]:
business['temp'] = business['name'].str.contains('Einstein Bros', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2167]:
business['temp'] = business['name'].str.contains('Pita Jungle', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2168]:
business['temp'] = business['name'].str.contains('Streets of New York', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2169]:
business['temp'] = business['name'].str.contains('Peter Piper Pizza', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2170]:
business['temp'] = business['name'].str.contains('Qdoba', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2171]:
business['temp'] = business['name'].str.contains('Papa Murphy', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2172]:
business['temp'] = business['name'].str.contains('McDonalds', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2173]:
business['temp'] = business['name'].str.contains(r'(?=.*Jack)(?=.*Box)',regex=True)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2174]:
business['temp'] = business['name'].str.contains('McDonalds', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2175]:
business['temp'] = business['name'].str.contains('McDonald', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2176]:
business['temp'] = business['name'].str.contains('El Pollo Loco', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2177]:
business['temp'] = business['name'].str.contains('Pei Wei', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2178]:
business['temp'] = business['name'].str.contains('Quiznos', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2179]:
business['temp'] = business['name'].str.contains('Firehouse Subs', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2180]:
business['temp'] = business['name'].str.contains('Del Taco', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2181]:
business['temp'] = business['name'].str.contains('Potbelly Sandwich Shop', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2182]:
business['temp'] = business['name'].str.contains('Long John Silver', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2183]:
business['temp'] = business['name'].str.contains('Pita Pit', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2184]:
business['temp'] = business['name'].str.contains('Schlotzsky', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2185]:
business['temp'] = business['name'].str.contains('Johnny Rockets', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2186]:
business['temp'] = business['name'].str.contains('Great Steak and Potato', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2187]:
business['temp'] = business['name'].str.contains('Auntie Anne', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2188]:
business['temp'] = business['name'].str.contains('Blaze Fast-Fire', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2189]:
business['temp'] = business['name'].str.contains('Blimpie', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2190]:
business['temp'] = business['name'].str.contains(r'(?=.*Church)(?=.*Chicken)',regex=True)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2191]:
business['temp'] = business['name'].str.contains(r'(?=.*Sonic)(?=.*Drive)',regex=True)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2192]:
business['temp'] = business['name'].str.contains('Burger King', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2193]:
business['temp'] = business['name'].str.contains('Dairy Queen', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2194]:
business['temp'] = business['name'].str.contains(r'(?=.*Carl)(?=.*Jr)',regex=True)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2195]:
business['temp'] = business['name'].str.contains(r'(?=.*Noodles)(?=.*Co)',regex=True)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2196]:
business['temp'] = business['name'].str.contains(r'(?=.*Olive)(?=.*Garden)',regex=True)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2197]:
business['temp'] = business['name'].str.contains('Applebee', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2198]:
business['temp'] = business['name'].str.contains('Starbucks', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2199]:
business['temp'] = business['name'].str.contains('Taco Bell', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2200]:
business['temp'] = business['name'].str.contains('Subway', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2201]:
business['temp'] = business['name'].str.contains('Chick-fil-A', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2202]:
business['temp'] = business['name'].str.contains('Dunkin', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2203]:
business['temp'] = business['name'].str.contains('Wendy', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2204]:
business['temp'] = business['name'].str.contains('Domino', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2205]:
business['temp'] = business['name'].str.contains('Panera', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2206]:
business['temp'] = business['name'].str.contains('Pizza Hut', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2207]:
business['temp'] = business['name'].str.contains('Chipotle', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2208]:
business['temp'] = business['name'].str.contains('KFC', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2209]:
business['temp'] = business['name'].str.contains('Arby', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2210]:
business['temp'] = business['name'].str.contains('Little Caesar', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2211]:
business['temp'] = business['name'].str.contains('Buffalo Wild Wings', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2212]:
business['temp'] = business['name'].str.contains('Panda Express', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2213]:
business['temp'] = business['name'].str.contains('IHOP', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2214]:
business['temp'] = business['name'].str.contains('Popeyes', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2215]:
business['temp'] = business['name'].str.contains('Texas Roadhouse', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2216]:
business['temp'] = business['name'].str.contains('Denny', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2217]:
business['temp'] = business['name'].str.contains('Outback Steakhouse', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2218]:
business['temp'] = business['name'].str.contains('Cracker Barrel', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2219]:
business['temp'] = business['name'].str.contains('Whataburger', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2220]:
business['temp'] = business['name'].str.contains('Red Lobster', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2221]:
business['temp'] = business['name'].str.contains('Jimmy John', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2222]:
business['temp'] = business['name'].str.contains('Cheesecake Factory', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2223]:
business['temp'] = business['name'].str.contains('Hardee', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2224]:
business['temp'] = business['name'].str.contains('Zaxby', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2225]:
business['temp'] = business['name'].str.contains('Five Guys', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2226]:
business['temp'] = business['name'].str.contains('Culver', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2227]:
business['temp'] = business['name'].str.contains('Waffle House', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2228]:
business['temp'] = business['name'].str.contains('Bojangles', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2229]:
business['temp'] = business['name'].str.contains('Wingstop', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2230]:
business['temp'] = business['name'].str.contains('Raising Cane', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2231]:
business['temp'] = business['name'].str.contains('TGI Friday', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2232]:
business['temp'] = business['name'].str.contains('Jersey Mike', regex=False)
business['chain'] = np.where(business['temp'] == True, 'Chain', business['chain'])

In [2233]:
business['cat'].value_counts()

Mexican                      509
American (New)               323
Pizza                        279
Burgers                      259
Sandwiches                   227
Chinese                      167
Cafe                         161
Italian                      115
Coffee & Tea                  70
Sushi                         67
Chicken                       60
American (Traditional)        58
Ice Cream & Frozen Yogurt     57
Diners                        51
Barbeque                      49
Greek                         48
Sports Bar                    47
Thai                          42
Cocktail Bars                 41
Burger                        40
Wine Bars                     38
Deli                          38
Steakhouses                   37
Vietnamese                    37
Indian                        29
Juice Bars & Smoothies        27
Japanese                      27
Asian Fusion                  25
Seafood                       22
Bagels                        20
Hot Dogs  

In [2234]:
business.head(20)

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,...,categories,cat,Fast Food,Bars,Gluten-Free,Vegetarian,Vegan,Pizza,temp,chain
5,rDMptJYWtnMhpQu_rRXHng,"""McDonald's""",NaN,"""719 E Thunderbird Rd""",Phoenix,AZ,85022,33.607070,-112.064382,1.0,...,Fast Food;Burgers;Restaurants,Burgers,True,False,False,False,False,False,False,Chain
6,1WBkAuQg81kokZIPMpn9Zg,"""Charr An American Burger Bar""",NaN,"""777 E Thunderbird Rd, Ste 107""",Phoenix,AZ,85022,33.607310,-112.063404,3.0,...,Burgers;Restaurants,Burgers,False,False,False,False,False,False,False,Non-Chain
8,iPa__LOhse-hobC2Xmp-Kw,"""McDonald's""",NaN,"""1635 E Camelback Rd""",Phoenix,AZ,85016,33.508765,-112.046240,3.0,...,Restaurants;Burgers;Fast Food,Burgers,True,False,False,False,False,False,False,Chain
12,QkG3KUXwqZBW18A9k1xqCA,"""Red Lobster""",NaN,"""2810 North 75th Ave""",Phoenix,AZ,85035,33.478735,-112.221379,2.5,...,American (Traditional);Restaurants;Seafood,Seafood,False,False,False,False,False,False,False,Chain
13,5XejqzaFmtkZMstJS5Iy-w,"""D'Lish Cafe""",NaN,"""503 W Thomas Rd""",Phoenix,AZ,85013,33.480301,-112.080586,4.0,...,Vegan;American (New);Restaurants;Sandwiches;Sa...,Cafe,False,False,False,False,True,False,False,Non-Chain
16,2v-8QQfMLX2PCz-0S6gISQ,"""Papa John's Pizza""",NaN,"""3441 W Northern Ave""",Phoenix,AZ,85051,33.552869,-112.133712,2.5,...,Pizza;Restaurants;Fast Food,Pizza,True,False,False,False,False,True,False,Chain
17,M3uV9Y3EDSpy9d4YwyNSAQ,"""Yakiramen""",NaN,"""10605 N 43rd Ave""",Phoenix,AZ,85029,33.582363,-112.149884,4.0,...,Nightlife;Japanese;Music Venues;Ramen;Arts & E...,Japanese,False,True,False,False,False,False,False,Non-Chain
30,U1ZVgF-kfkvv_rcoe0RglQ,"""Pho Noodles""",NaN,"""3417 N 7th Ave""",Phoenix,AZ,85013,33.488760,-112.082402,4.5,...,Restaurants;Noodles;Vietnamese,Vietnamese,False,False,False,False,False,False,False,Non-Chain
31,FfI9FOaQqzUxixw6_glt3Q,"""Affinito's Bistro""",NaN,"""3655 W Anthem Way, Ste C-137""",Phoenix,AZ,85086,33.864964,-112.138820,3.0,...,Restaurants;Italian,Italian,False,False,False,False,False,False,False,Non-Chain
32,3b4efqz06QrLQ_w2xLc4pA,"""Olive Garden Italian Restaurant""",NaN,"""10223 N Metro Parkway E""",Phoenix,AZ,85051,33.580302,-112.119653,3.0,...,Restaurants;Italian,Italian,False,False,False,False,False,False,False,Chain


#### Use the code below to interrogate categories

In [2235]:
i = 'Non-Chain'#write in what category you are interested in

In [2236]:
bizzy = business.loc[business['chain']==i]
pd.set_option('display.max_row', 100)
bizzy['name'].value_counts()

"Julioberto's"                               9
"Barro's Pizza"                              7
"Paradise Bakery & Cafe"                     6
"Port of Subs"                               6
"Federico's Mexican Food"                    6
"My Big Fat Greek Restaurant"                5
"Fired Pie"                                  5
"First Watch - Breakfast, Brunch & Lunch"    5
"Chuy's Mesquite Broiler"                    5
"America's Taco Shop"                        5
"Wildflower Bread Company"                   4
"Armando's Mexican Food"                     4
"Paradise Bakery"                            4
"Matt's Big Breakfast"                       4
"Los Dos Molinos"                            4
"Hungry Howie's Pizza"                       4
"Sweet Tomatoes"                             4
"Humble Pie"                                 4
"Taco Mich"                                  4
"Cafe Rio"                                   4
"Ray's Pizza"                                4
"Souper Salad

In [2237]:
bizzy.head(20)

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,...,categories,cat,Fast Food,Bars,Gluten-Free,Vegetarian,Vegan,Pizza,temp,chain
6,1WBkAuQg81kokZIPMpn9Zg,"""Charr An American Burger Bar""",NaN,"""777 E Thunderbird Rd, Ste 107""",Phoenix,AZ,85022,33.607310,-112.063404,3.0,...,Burgers;Restaurants,Burgers,False,False,False,False,False,False,False,Non-Chain
13,5XejqzaFmtkZMstJS5Iy-w,"""D'Lish Cafe""",NaN,"""503 W Thomas Rd""",Phoenix,AZ,85013,33.480301,-112.080586,4.0,...,Vegan;American (New);Restaurants;Sandwiches;Sa...,Cafe,False,False,False,False,True,False,False,Non-Chain
17,M3uV9Y3EDSpy9d4YwyNSAQ,"""Yakiramen""",NaN,"""10605 N 43rd Ave""",Phoenix,AZ,85029,33.582363,-112.149884,4.0,...,Nightlife;Japanese;Music Venues;Ramen;Arts & E...,Japanese,False,True,False,False,False,False,False,Non-Chain
30,U1ZVgF-kfkvv_rcoe0RglQ,"""Pho Noodles""",NaN,"""3417 N 7th Ave""",Phoenix,AZ,85013,33.488760,-112.082402,4.5,...,Restaurants;Noodles;Vietnamese,Vietnamese,False,False,False,False,False,False,False,Non-Chain
31,FfI9FOaQqzUxixw6_glt3Q,"""Affinito's Bistro""",NaN,"""3655 W Anthem Way, Ste C-137""",Phoenix,AZ,85086,33.864964,-112.138820,3.0,...,Restaurants;Italian,Italian,False,False,False,False,False,False,False,Non-Chain
41,_eKvk2GoGKb3CoYolHgRlg,"""Fresko A Mediterranean Kitchen""",NaN,"""5033 E Elliot Rd""",Phoenix,AZ,85044,33.347893,-111.976822,4.5,...,Greek;Vegetarian;Sandwiches;Mediterranean;Rest...,Greek,False,False,False,True,False,False,False,Non-Chain
47,rSCevr5tQJTxPzMIq3n5Gw,"""Yoshi's Asian Grill""",NaN,"""3800 E Sky Harbor Blvd""",Phoenix,AZ,85034,33.435621,-112.009546,2.5,...,Restaurants;Asian Fusion,American (New),False,False,False,False,False,False,False,Non-Chain
63,X2oxkKB-wlu8xV0P8fbfiQ,"""Mo's Chicken""",NaN,"""3505 W Thomas Rd""",Phoenix,AZ,85019,33.480049,-112.134702,3.0,...,Restaurants;Chicken Shop,Chicken,False,False,False,False,False,False,False,Non-Chain
86,frCxZS7lPhEnQRJ3UY6m7A,"""La Santisima""",NaN,"""1919 N 16th St""",Phoenix,AZ,85006,33.469201,-112.047393,4.0,...,Vegetarian;Nightlife;Mexican;Bars;Vegan;Restau...,Mexican,False,True,False,True,True,False,False,Non-Chain
90,l0WauZ92kRg9bu2oxHj4TQ,"""Aiello's East Coast Italian""",NaN,"""777 E Thunderbird Rd, Ste 100""",Phoenix,AZ,85022,33.607105,-112.063427,4.0,...,Restaurants;Italian,Italian,False,False,False,False,False,False,False,Non-Chain
